# Objective function `the best place for the company to grow`

##### Objective function ->

X (var independient): Val_by_place Each place will be assigned with a value in each component (9 x 1)

Constants: 
     - Total personal (9 x 1)
     - Valoration of the contribution to grow by role (9 x 1)
     - Rel_personal_Val_place represent the relation between condition and personal (9 x 9)

Any places will be valuated with the next equation:

             `Constants`                      `Independient Variable`

F   =    (   Personal   *   Val_to_grow   *   Rel_personal_Val_place  )    *    Val_by_place

In [1]:
import numpy as np
import json
from pymongo import MongoClient
import pandas as pd

import sys
sys.path.insert(0, 'src/')

from funobjetivo import valor


In [2]:
# Example of using function objective
Val_by_place={"other Design companies":0.1,
         "kindergarden":0.2,
         "success startup":0.5,
         "Starbucks":1,
         "Airport_or_train_station":2,
         "place_to party":2,
         "basqueball court":1,
         "no_2km_old_company":0.8,
         "vegan_restaurant":0.5}

valor(**Val_by_place)

2255.75

# Getting data

## Other design companies

In [3]:
# Connect and get the data --> Categories
url_mongo="mongodb://localhost:27017"
client = MongoClient(url_mongo)
db = client.get_database("companies")
result=list(db.companies.find({},{"category_code":1,"_id":0}))
categories=[e['category_code'] for e in result]

In [4]:
# Count the number of design comanies and include Web 
categorias= pd.DataFrame(categories)
categorias=categorias.rename(columns={0:"categoria"})
len(categorias.categoria.value_counts())
# -->  41 categories
len(categorias.categoria[categorias.categoria=="design"]) 
# --> just 4 companies!!
len(categorias.categoria[categorias.categoria=="web"]) 
# --> 3787.. is web companies equivalent to design 
#     companies in this context?

3787

In [5]:
# Getting the ubication of the offices of design and web companies
query={"$or":[{"category_code":"design"},{"category_code":"web"}]}
offices=list(db.companies.find(query,{"name":1,"offices":1,"_id":0}))
offices=pd.DataFrame(offices)
offices=offices.explode("offices")
offices

,name,offices
0,Wetpaint,"{'description': '', 'address1': '710 - 2nd Ave..."
0,Wetpaint,"{'description': '', 'address1': '270 Lafayette..."
1,Postini,"{'description': None, 'address1': '959 Skyway ..."
2,Geni,"{'description': 'Headquarters', 'address1': '9..."
3,Fox Interactive Media,"{'description': '', 'address1': '407 N Maple D..."
...,...,...
3786,iGuiders,"{'description': '', 'address1': '23811 Chagrin..."
3787,Zoomission,NaN
3788,Eazeeloans,"{'description': 'Apex Finance & Marketing', 'a..."
3789,DocASAP,"{'description': '', 'address1': '115 5th Ave',..."


In [6]:
# Extracting Lat an Long
def json_column(row):
    of=row.offices
    if type(of)==float or of["latitude"] is None:
        return (row['name'],"theres is no data ",None, None)
    return (row['name'],"success",of["latitude"] ,of["longitude"])

offices_coor_name = offices.apply(json_column, axis=1, result_type='expand')
offices_coor_name# = pd.concat([offices.name,offices_coor])

,0,1,2,3
0,Wetpaint,success,47.603122,-122.333253
0,Wetpaint,success,40.723731,-73.996431
1,Postini,success,37.506885,-122.247573
2,Geni,success,34.090368,-118.393064
3,Fox Interactive Media,success,34.076179,-118.394170
...,...,...,...,...
3786,iGuiders,theres is no data,NaN,NaN
3787,Zoomission,theres is no data,NaN,NaN
3788,Eazeeloans,theres is no data,NaN,NaN
3789,DocASAP,theres is no data,NaN,NaN


In [7]:
offices_coor_name=offices_coor_name.rename(columns={0:'name',1:'State_coor',2:'latitude',3:'longitud'} )
offices_coor_name=offices_coor_name[offices_coor_name['State_coor']=="success"] 
# --> 2311 companies with coordenates

In [8]:
# Setting the format for MongoDB index
def coor_jgeo(row):
     return {'type':'Point','coordinates':[row['longitud'],row['latitude']]}

offices_coor_name['design_coor']=offices_coor_name.apply(coor_jgeo,axis=1)

In [9]:
offices_coor_name.to_json('output/design.json',orient="records")

## Success startup

In [10]:
# Query for getting the rounds data
query={"funding_rounds.0.raised_amount":{"$gt":0}}

fund=list(db.companies.find(query,{"name":1,"funding_rounds":1,"_id":0}))
fund=pd.DataFrame(fund)
fund=fund.explode("funding_rounds") # several rounds...

In [11]:
# Creating columns or Raised amount and currency
def extract_round(var):
    return var['raised_amount']

def extract_currency(var):
    return var['raised_currency_code']

fund['val_round']=fund.funding_rounds.apply(extract_round)
fund['val_currency']=fund.funding_rounds.apply(extract_currency)

In [12]:
# Currencies of the Rounds
fund.groupby(['val_currency']).count()

,name,funding_rounds,val_round
val_currency,,,
CAD,43,43,43
EUR,336,336,336
GBP,151,151,151
JPY,12,12,12
SEK,1,1,1
USD,10145,10145,10003


In [13]:
# Creating the rounds in USD--> (average currency exchange last 5Y)
def calculate_usd(row):
    if row['val_round'] is None or row['val_currency'] is None:
        return 0
    
    currecy_equi={'CAD':0.8,'EUR':1.2,'GBP':1.5,'JPY':0.01,'SEK':0.15,'USD':1}
    val=row['val_round'] * currecy_equi[row['val_currency']]

    return val

fund['Val_round_USD']=fund.apply(calculate_usd,axis=1)

Here we have the startups that have funds greater than 1 million dollars

In [14]:
fund=fund[['name','Val_round_USD']].groupby(['name']).sum()
fund=fund.loc[fund.Val_round_USD>1e6]
# --> 3686 companies with more than a millon raised

##### Now find the Lat Long

In [15]:
query={"funding_rounds.0.raised_amount":{"$gt":0}}

ss_point=list(db.companies.find(query,{"name":1,"offices":1,"_id":0}))
ss_point=pd.DataFrame(ss_point)
ss_point=ss_point.explode('offices')

In [16]:
ss_coor=ss_point.apply(json_column, axis=1, result_type='expand')
ss_coor=ss_coor.rename(columns={0:'name',1:'State_coor',2:'latitude',3:'longitud'} )
ss_coor.State_coor.value_counts()

success               3815
theres is no data     2193
Name: State_coor, dtype: int64

In [17]:
ss_coor=ss_coor.loc[ss_coor.State_coor=="success"]
# --> 3815 points of succesful companies
ss_coor.columns

Index(['name', 'State_coor', 'latitude', 'longitud'], dtype='object')

In [18]:
# Setting the format for MongoDB index and export json
ss_coor['success_coor']=ss_coor.apply(coor_jgeo,axis=1)
ss_coor.to_json('output/success.json',orient="records")

## Old Companies

In [19]:
# Query for getting the founded year
query={"founded_year":{"$lt":2010}}

old=list(db.companies.find(query,{"name":1,"offices":1,"_id":0}))
old=pd.DataFrame(old)
old=old.explode("offices") # several offices
old

,name,offices
0,AdventNet,"{'description': 'Headquarters', 'address1': '4..."
1,Wetpaint,"{'description': '', 'address1': '710 - 2nd Ave..."
1,Wetpaint,"{'description': '', 'address1': '270 Lafayette..."
2,Zoho,"{'description': 'Headquarters', 'address1': '4..."
3,Omnidrive,"{'description': '', 'address1': 'Suite 200', '..."
...,...,...
12985,Oriact,"{'description': '', 'address1': 'LÃ¤rchenweg 7..."
12986,Adhunk,"{'description': 'Indian Office', 'address1': '..."
12987,AfterLogic,"{'description': 'Livingston', 'address1': 'P.O..."
12988,Contemporary Computer Services,"{'description': '', 'address1': '200 Knickerbo..."


In [20]:
old=old.apply(json_column, axis=1, result_type='expand')
old=old.rename(columns={0:'name',1:'State_coor',2:'latitude',3:'longitud'} )
old.State_coor.value_counts()

success               9039
theres is no data     6725
Name: State_coor, dtype: int64

In [21]:
old=old.loc[old.State_coor=="success"]
# --> 9039 points of old companies (<2010!!)
old.columns

Index(['name', 'State_coor', 'latitude', 'longitud'], dtype='object')

In [22]:
# Setting the format for MongoDB index and export it
old['success_coor']=old.apply(coor_jgeo,axis=1)
old.to_json('output/old.json',orient="records")

## Importing from API google maps

In [23]:
import requests 
import json
import os 

from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.insert(0, 'src/')
import apiquery # module with the db data

##### --> Cities to import 

We are going to limit the search to specific cities:

* Madrid (40.416977, -3.703827)
* New York (40.7142715 ,-74.0059662)
* Berlin (52.5243683, 13.4105301)
* Seatle (47.6062088, -122.3320694)

In [24]:
apiquery.genera_bd() # <-- Función que consulta el API de google y crea archivos locales para consulta

True

In [1]:
# The research is by city:
import sys
sys.path.insert(0, 'src/')
import research_mad

point_madrid=(40.416977, -3.703827)
point_sevilla=(37.3828300,-5.9731700)
#research_mad.val_design(point_madrid)
#research_mad.val_succ(point_sevilla)

research_mad.val_old(point_sevilla)

#Val_by_place={"other Design companies":val_design_madrid(point_madrid),
#         "kindergarden":val_kindergarden_madrid(point_madrid),
#         "success startup":val_succ_madrid(point_madrid),
#         "Starbucks":val_starbucks_madrid(point_madrid),
#         "Airport_or_train_station":val_airport_madrid(point_madrid),
#         "place_to party":val_party_madrid(point_madrid),
#         "basqueball court":val_basquet_madrid(point_madrid),
#         "no_2km_old_company":val_old_madrid(point_madrid),
#         "vegan_restaurant":val_vegetariano_madrid(point_madrid)}


0.9815641667837387

In [2]:
research_mad.f_haversine(point_sevilla,point_madrid)

1818068.5111826349